## Photometry on MIRI images

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import warnings
import subprocess
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning
from astropy.table import Table
import astropy.units as u
from astropy.stats import SigmaClip

import stpsf
from photutils.aperture import EllipticalAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

warnings.simplefilter("ignore", category=FITSFixedWarning)


Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table.columns)

Now let's choose a galaxy, read its aperture data and overplot it!

In [ ]:
def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def adjust_aperture(galaxy_id, filter, survey, obs, output_folder, save_plot=False):
    
    # --- Load the FITS table ---
    #table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    #aperture_table = Table.read(table_path)
    table_path =  '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'
    df = pd.read_csv(table_path)
    
    # --- Select the galaxy by ID ---
    #row = aperture_table[aperture_table['ID'] == galaxy_id][0]
    galaxy_id = int(galaxy_id)
    row = df[df['ID'] == galaxy_id].iloc[0]

    # --- Read in rotation angle of MIRI FITS file ---
    angle_file = '/home/bpc/University/master/Red_Cardinal/rotation_angles.json'
    with open(angle_file, "r") as f:
        angles = json.load(f)
    angle = angles[f"angle_{survey}{obs}"]
    
    # --- Read WCS from NIRCam image ---
    nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{galaxy_id}_F444W_cutout.fits"
    nircam_data, nircam_wcs = load_cutout(nircam_path)

    # --- Convert NIRCam pixel coordinates to sky ---
    sky_coord = nircam_wcs.pixel_to_world(row['Apr_Xcenter'], row['Apr_Ycenter'])

    # --- Open MIRI cutout image ---
    miri_path = f"/home/bpc/University/master/Red_Cardinal/cutouts/{galaxy_id}_{filter}_cutout_{survey}{obs}.fits"
    miri_data, miri_wcs = load_cutout(miri_path)

    # --- Convert sky coords to MIRI pixel coordinates ---
    miri_x, miri_y = miri_wcs.world_to_pixel(sky_coord)

    # --- Create elliptical region in MIRI pixel space ---
    nircam_scale = 0.03    # arcsec/pixel
    miri_scale = 0.11092  # arcsec per pixel
    
    # arcsec/pixel
    scale_factor = nircam_scale / miri_scale
    
    # --- Specify parameters for the ellips ---
    width = row['Apr_A'] * scale_factor
    height = row['Apr_B'] * scale_factor
    theta = -row['Apr_Theta']
    theta_new = ((theta - angle) % 180) * u.deg
    
    # --- Create region file and check if folder exists ---
    os.makedirs(output_folder, exist_ok=True)
    reg_file = os.path.join(output_folder, f'regions/{galaxy_id}_{survey}{obs}_aperture.reg') 
    
    # --- Write to DS9-compatible region file ---
    with open(reg_file, "w") as fh:
        fh.write("# Region file format: DS9 version 4.1\n")
        fh.write("global color=red dashlist=8 3 width=2 font=\"helvetica 10 normal\" "
                "select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        fh.write("image\n")
        fh.write(f"ellipse({miri_x:.2f},{miri_y:.2f},{width:.2f},{height:.2f},{theta_new:.2f})\n")

    if save_plot:
        
        # --- Clean and prepare the MIRI data for plotting ---
        miri_clean = np.copy(miri_data)
        finite_vals = miri_clean[np.isfinite(miri_clean)].flatten()
        
        # Sort and get lowest 80% values
        sorted_vals = np.sort(finite_vals)
        cutoff_index = int(0.8 * len(sorted_vals))
        background_vals = sorted_vals[:cutoff_index]
        background_mean = np.mean(background_vals)

        # Replace NaNs or infs with background mean
        miri_clean[~np.isfinite(miri_clean)] = background_mean

        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(miri_clean, origin='lower', cmap='gray', vmin=np.percentile(miri_clean, 5), vmax=np.percentile(miri_clean, 99))

        # Original ellipse (without rotation correction)
        ellipse_original = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta,  # Just the original θ!
            edgecolor='red',
            facecolor='none',
            lw=2,
            label='Original Ellipse'
        )
        ax.add_patch(ellipse_original)

        ellipse = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta_new.to_value(u.deg),
            edgecolor='blue',
            linestyle='--',  # maybe dashed to differentiate
            facecolor='none',
            lw=2,
            label='Rotated Ellipse'
        )
        ax.add_patch(ellipse)
        
        
        
        ax.set_title(f"Galaxy {galaxy_id} - {filter} ({survey}{obs})")
        ax.set_xlim(miri_x - 30, miri_x + 30)
        ax.set_ylim(miri_y - 30, miri_y + 30)
        ax.legend(loc='upper right')
        
        # Save figure
        png_path = os.path.join(output_folder, f'masks/{galaxy_id}_{survey}{obs}_aperture_overlay.png')
        plt.savefig(png_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
        
    # Collect modified aperture data
    aperture_info = {
    'Flux': np.nan,           # You don't have these yet
    'Flux_Err': np.nan,        
    'Image_Err': np.nan,       
    'Flux_BKG': np.nan,        
    'AB_Mag': np.nan,          
    'Flux_BKG_Err': np.nan,    
    'N_PIX': np.nan,           
    'Apr_A': width,            # Rescaled aperture
    'Apr_B': height,
    'Apr_Xcenter': miri_x,
    'Apr_Ycenter': miri_y,
    'Apr_Theta': theta_new.to_value(u.deg),
    'ID': galaxy_id
    }
    
    return aperture_info
    

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '20397'
filter = 'F770W'
survey_obs = 'cweb2'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()

# Section to perform the actual photometry

Imports

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import warnings
import subprocess
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning
from astropy.table import Table
import astropy.units as u
from astropy.stats import SigmaClip

import stpsf
from photutils.aperture import EllipticalAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

warnings.simplefilter("ignore", category=FITSFixedWarning)


First, let's set some parameters straight:

In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts/"
output_table = '/home/bpc/University/master/Red_Cardinal/photometry/miri_photometry.ecsv'

filters = ['F770W', 'F1800W']  # Your two filters
psf_sampling = 0.11  # arcsec/pixel for MIRI imaging

pixscale_arcsec = 0.11092  # arcsec per pixel
pix_area_sr = 2.89208962133982e-13  # from MIRI header

box_size = 50
filter_size = 3

sigma_clip = SigmaClip(sigma=3.0)
bkg_estimator = MedianBackground()

Functions

In [ ]:
# --- Functions ---

def get_psf(filter_name):
    """Generate MIRI PSF using WebbPSF."""
    psf_file = f'/home/bpc/University/master/Red_Cardinal/WebbPSF/PSF_MIRI_{filter_name}.fits'

    with fits.open(psf_file) as psf:
        psf_data = psf[0].data
    return psf_data


def calculate_aperture_correction(psf_data, aperture_params):
    """Calculate aperture correction for given PSF and aperture."""
    y, x = np.indices(psf_data.shape)
    center = np.array(psf_data.shape) / 2.0

    aperture = EllipticalAperture(
        positions=(center[1], center[0]),
        a=aperture_params['a'],
        b=aperture_params['b'],
        theta=aperture_params['theta']
    )

    total_flux = np.sum(psf_data)

    phot_table = aperture_photometry(psf_data, aperture)
    flux_in_aperture = phot_table['aperture_sum'][0]

    correction_factor = total_flux / flux_in_aperture
    return correction_factor

def background_model(data):
    """Model background using Background2D."""
    bkg = Background2D(
        data,
        box_size=box_size,
        filter_size=filter_size,
        sigma_clip=sigma_clip,
        bkg_estimator=bkg_estimator
    )
    return bkg.background, bkg.background_rms

def flux_to_abmag(flux):
    """Convert flux in Jy to AB magnitude."""
    # For JWST/MIRI, you typically get flux in MJy/sr — make sure to convert accordingly
    flux_jy = flux  # assuming flux is already in Jy
    if flux_jy > 0:
        mag = -2.5 * np.log10(flux_jy) + 8.90
    else:
        mag = np.nan
    return mag

Main workflow

In [ ]:
# --- Main flow ---

# Output table
photometry_results = []

# Let's try with galaxy 9871
galaxy_id = '9871'

for filter_name in filters:
    fits_path = os.path.join(cutouts_folder, f'{galaxy_id}_{filter_name}_cutout_primer004.fits')

    with fits.open(fits_path) as hdul:
        data = hdul[1].data
    
    # Background subtraction
    background, background_rms = background_model(data)
    data_bkgsub = data - background

    # --- Define aperture ---
    aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv'
    df = pd.read_csv(aperture_table)
    row = df[df['ID']==int(galaxy_id)]
    
    miri_x = row['Apr_Xcenter'].values[0] # centre position in X
    miri_y = row['Apr_Ycenter'].values[0]  # centre position in Y
    width = row['Apr_A'].values[0]  # major axis (pixels)
    height = row['Apr_B'].values[0]  # minor axis (pixels)
    theta = row['Apr_Theta'].values[0] * u.deg

    aperture = EllipticalAperture(
        (miri_x, miri_y),
        a=width / 2,
        b=height / 2,
        theta=theta.to_value(u.rad)
    )

    # --- Photometry ---
    phot_table = aperture_photometry(data_bkgsub, aperture, method='exact')
    flux = phot_table['aperture_sum'][0]

    print('Flux: ', flux)
    
    # Background flux inside aperture
    n_pix = np.pi * (width/2) * (height/2)
    background_per_pixel = np.median(background)
    flux_bkg = background_per_pixel * n_pix

    print('Background Flux: ', flux_bkg)
    
    # Uncertainty estimation
    flux_uncertainty = np.sqrt(n_pix) * np.median(background_rms)
    image_err = np.median(background_rms)  # per pixel

    print('Background RMS: ', np.median(background_rms))
    
    # Aperture correction
    psf_data = get_psf(filter_name)
    aperture_params = {
        'a': width / 2,
        'b': height / 2,
        'theta': theta.to_value(u.rad)
    }
    correction_factor = calculate_aperture_correction(psf_data, aperture_params)
    
    print('Correction Factor: ', correction_factor)

    flux_corrected = flux * correction_factor
    flux_bkg_corrected = flux_bkg * correction_factor
    flux_uncertainty_corrected = flux_uncertainty * correction_factor

    # ---> MJy/sr to Jy conversion
    flux_jy = flux_corrected * pix_area_sr * 1e6  # MJy/sr × sr × (1e6)

    # AB magnitude
    ab_mag = flux_to_abmag(flux_jy)

    # --- Save result ---
    photometry_results.append({
        'Flux': flux_corrected,
        'Flux_Err': flux_uncertainty_corrected,
        'Image_Err': image_err,
        'Flux_BKG': flux_bkg_corrected,
        'AB_Mag': ab_mag,
        'Flux_BKG_Err': flux_uncertainty_corrected,
        'N_PIX': n_pix,
        'Apr_A': width/2,
        'Apr_B': height/2,
        'Apr_Xcenter': miri_x,
        'Apr_Ycenter': miri_y,
        'Apr_Theta': theta.to_value(u.deg),
        'ID': int(galaxy_id)
    })

# Save full table
phot_table = Table(rows=photometry_results, names=(
    'Flux','Flux_Err','Image_Err','Flux_BKG','AB_Mag',
    'Flux_BKG_Err','N_PIX','Apr_A','Apr_B','Apr_Xcenter',
    'Apr_Ycenter','Apr_Theta','ID'
))
phot_table.write(output_table, format='ascii.ecsv', overwrite=True)

print(f"Saved final photometry results to {output_table}")


Check the header of the FITS files:

In [ ]:
primer003 =  '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER_003/'
primer004 =  '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER_004/'
cweb1 =   '/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web_1/'
cweb2 =   '/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web_2/'
nircam = '/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/'

fits_file = os.path.join(nircam, '7102_F444W_cutout.fits')

with fits.open(fits_file) as hdul:
    hdr = hdul[1].header  # usually the science data is in extension 1, but check
    print(repr(hdr))